In [20]:
import matplotlib.pyplot as plt
import numpy as np
import os
from oiffile import imread
from PIL import Image
import random
import skimage
from skimage.transform import resize
import matplotlib
import glob
import torch
import sys
import scipy.io as sio
import matplotlib.patches as patches
from skimage.measure import label,regionprops
from matplotlib_scalebar.scalebar import ScaleBar
from skimage.transform import rescale
from sklearn.metrics import roc_curve, auc
import os, os.path

In [21]:
def make_training_set(indexes):
    train_images = []
    for i in indexes:
        train_images.append(collman[:,i])
    return train_images

medianTotal= np.double(np.load("../../datasets/vermisB/medianTotal.npy"))
meanTotal= np.double(np.load("../../datasets/vermisB/meanTotal.npy"))

medianclipped0 = np.maximum(medianTotal[0],np.max(medianTotal[0])*0.6)
medianclipped1 = np.maximum(medianTotal[1],np.max(medianTotal[1])*0.6)
medianclipped2 = np.maximum(meanTotal[2],np.max(meanTotal[2])*0.6)

In [23]:
with open("../../datasets/vermisB/imagesUsed-vermisB.txt") as fin:
    lines = fin.readlines()
    
for num in range(0,len(lines)):
    fname = lines[num]
    fname = fname[:12]
    v = fname[10:]
    n = fname[5:9]
    
    z_slice = int(v)

    fdir = r'E:\pcp2cre_syptom_568_mglur1_1to200_647_1to250_vgat_1to200_488_1to250\FV10__20190508_202030_vermisB\Track%s' % (n)
    
    im_name = 'Image%s_01.oib' % (n)
    image = imread(os.path.join(fdir,im_name))
    file_name= 'Track%s' % (n)
    
    image_save = np.double(image[:,z_slice,:,:].transpose(1,2,0))
    image_save[:,:,0] = image_save[:,:,0]/medianclipped0[z_slice]
    image_save[:,:,1] = image_save[:,:,1]/medianclipped1[z_slice]
    image_save[:,:,2] = image_save[:,:,2]/medianclipped2[z_slice]

    image_save = resize(image_save,(400,400), order=1, preserve_range=True)

    h=400
    w=400
    tol=30

    UL=(image_save[0:int(h/2+tol),0:int(w/2+tol),:])       
    UR=(image_save[0:int(h/2+tol),int(w/2-tol):w,:])
    LL=(image_save[int(h/2-tol):h,0:int(w/2+tol),:])
    LR=(image_save[int(h/2-tol):h,int(w/2-tol):w,:])

    smallPictures=[UL,UR,LL,LR]

    collman = np.transpose((np.stack(smallPictures,axis=3)),(2,3,0,1))
    train_images = make_training_set(range(0,4))

    smallPicturesNames=['UL','UR','LL','LR']

    for j in range(0,4): 
        imageStr="../../datasets/vermisB/NEWtrainingImages/%s_%s_%s_image.npy" % (file_name,v,smallPicturesNames[j])
        np.save(imageStr, train_images[j])